# Case
## Imports

In [1]:
import cadquery as cq
from cadquery import exporters

from jupyter_cadquery import (
    versions,
    show, PartGroup, Part, 
    get_viewer, close_viewer, get_viewers, close_viewers, open_viewer, set_defaults, get_defaults, open_viewer,
    get_pick,
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay

enable_replay(False)

set_defaults(
    cad_width=640, 
    height=480, 
)

print()
versions()

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay

jupyter_cadquery  3.1.0rc4
cad_viewer_widget 1.3.2
OCP               7.5.3


## Viewer

In [2]:
cv = open_viewer("Assembly", cad_width=850, height=1240)
set_defaults(viewer="Assembly")

## Constants

In [3]:
BOTTOM_PLATE_HEIGHT = 1.0
PCB_BOTTOM_SPACING = 2.4
PCB_HEIGHT = 1.6
SWITCH_PLATE_HEIGHT = 1.2
OLED_SPACING = 8
ACRYLIC_GLASS_HEIGHT = 3

SWITCH_BORDER_HEIGHT = 5

HOLES = [((96.9, 107.88), 2, 0), 
         ((175.28, 86.28), 2, 0), 
         ((135.05, 37), 4, BOTTOM_PLATE_HEIGHT + 4)]

## Helper Functions

In [4]:
def createCaseOutline():
    return cq.Workplane("XY")\
    .moveTo(78.77, -42.15)\
    .lineTo(78.77, -90.9)\
    .radiusArc((75.91, -96.0), 8)\
    .lineTo(70.57, -99.08)\
    .radiusArc((68.98, -105.04), -4.3)\
    .lineTo(82.05, -127.54)\
    .radiusArc((88.00, -129.13), -4.3)\
    .radiusArc((118.04, -116.55), 79.9)\
    .lineTo(176.41, -116.55)\
    .radiusArc((179.39, -113.61), -2.9)\
    .lineTo(179.39, -110.34)\
    .radiusArc((182.34, -107.39), 2.9)\
    .lineTo(194.62, -107.39)\
    .radiusArc((197.6, -104.45), -2.9)\
    .lineTo(197.6, -51.11)\
    .radiusArc((194.66, -48.13), -2.9)\
    .lineTo(182.34, -48.14)\
    .radiusArc((179.4, -45.16), 2.9)\
    .lineTo(179.4, -41.12)\
    .radiusArc((177.15, -37.56), -3.7)\
    .radiusArc((103.15, -37.15), -104.25)\
    .lineTo(83.77, -37.15)\
    .radiusArc((78.77, -42.15), -5)\
    .close()

def createBottomPlate():
    return createCaseOutline()\
        .faces()\
        .offset2D(-3)\
        .extrude(BOTTOM_PLATE_HEIGHT)

def createDisplayCover():
    return cq.Workplane("XY")\
    .moveTo(78.77, -42.15)\
    .lineTo(78.77, -90.9)\
    .lineTo(103.14, -90.9)\
    .lineTo(103.14, -37.15)\
    .lineTo(83.77, -37.15)\
    .radiusArc((78.77, -42.15), -5)\
    .close()\
    .faces()\
    .extrude(OLED_SPACING + ACRYLIC_GLASS_HEIGHT)\
    .translate((0, 0, BOTTOM_PLATE_HEIGHT + PCB_BOTTOM_SPACING + PCB_HEIGHT))

def extrudeDisplayCover(case):
    workplane = case.faces(">Z").workplane().moveTo(78.77, -42.15)\
        .lineTo(78.77, -90.9)\
        .lineTo(103.15, -90.9)\
        .lineTo(103.15, -37.15)\
        .lineTo(83.77, -37.15)\
        .radiusArc((78.77, -42.15), -5)\
        .close()\
        .workplane(offset=OLED_SPACING + ACRYLIC_GLASS_HEIGHT - SWITCH_BORDER_HEIGHT)
    return workplane.moveTo(82.77, -42.15)\
    .lineTo(82.77, -85.9)\
    .lineTo(103.15, -85.9)\
    .lineTo(103.15, -39.15)\
    .lineTo(83.77, -39.15)\
    .radiusArc((82.77, -42.15), -5)\
    .close()\
    .loft(combine=True)

def cutBottomPlate(case):
    return case.cut(createBottomPlate())

def cutSwitchHoles(case):
    zOffset = BOTTOM_PLATE_HEIGHT + PCB_BOTTOM_SPACING + PCB_HEIGHT
    cutout = cq.importers.importDXF('switch_outline.dxf', tol=1e-3)\
        .wires()\
        .toPending()\
        .faces()\
        .extrude(SWITCH_PLATE_HEIGHT)\
        .translate((0, 0, zOffset))
    return case.cut(cutout)

def cutKeycapsBorder(case):
    zOffset = BOTTOM_PLATE_HEIGHT + PCB_BOTTOM_SPACING + PCB_HEIGHT + SWITCH_PLATE_HEIGHT
    cutout = cq.importers.importDXF('keycaps_outline.dxf', tol=1e-3)\
        .wires()\
        .toPending()\
        .faces()\
        .extrude(SWITCH_BORDER_HEIGHT + 5)\
        .translate((0, 0, zOffset))
    return case.cut(cutout)

def cutPcb(case):
    zOffset = BOTTOM_PLATE_HEIGHT
    cutout = cq.importers.importDXF('pcb_outline.dxf', tol=1e-3)\
        .wires()\
        .toPending()\
        .faces()\
        .extrude(PCB_BOTTOM_SPACING + PCB_HEIGHT)\
        .translate((0, 0, zOffset))
    return case.cut(cutout)

def cutMcu(case):
    zOffset = BOTTOM_PLATE_HEIGHT + PCB_BOTTOM_SPACING + PCB_HEIGHT
    workplane = case.faces("<Z").workplane()
    vv = workplane.moveTo(82.78,88.81)\
        .lineTo(82.78, 96.18)\
        .lineTo(101.51, 96.18)\
        .lineTo(101.51, 88.81)\
        .close()\
        .extrude(-1, combine=False)\
        .translate((0, 0, zOffset))
    bb = workplane.moveTo(82.78,82.42)\
        .lineTo(82.78, 88.81)\
        .lineTo(101.51, 88.81)\
        .lineTo(101.51, 82.42)\
        .close()\
        .extrude(-5.5, combine=False)\
        .translate((0, 0, zOffset))
    
    nn = workplane.moveTo(82.78,42.26)\
        .lineTo(82.78, 82.42)\
        .lineTo(101.51, 82.42)\
        .lineTo(101.51, 42.26)\
        .close()\
        .extrude(-10,combine=False)\
        .translate((0, 0, zOffset))
    
    return case.cut(vv).cut(bb).cut(nn)

def cutAcrylicGlassWindow(case):
    zOffset = BOTTOM_PLATE_HEIGHT + PCB_BOTTOM_SPACING + PCB_HEIGHT +SWITCH_PLATE_HEIGHT + OLED_SPACING
    cutout = case.faces("<Z").workplane()\
        .moveTo(86.38,48.14)\
        .lineTo(86.38, 75.14)\
        .lineTo(98.38, 75.14)\
        .lineTo(98.38, 48.14)\
        .close()\
        .extrude(-ACRYLIC_GLASS_HEIGHT, combine = False)\
        .translate((0, 0, zOffset))
    return case.cut(cutout)

def cutAudioJack(case, rightSide):
    jackDiameter = 2.5
    shieldPcbHeight = 0.6
    zOffset = BOTTOM_PLATE_HEIGHT + PCB_BOTTOM_SPACING + jackDiameter - shieldPcbHeight
    yOffset = -83.81 if rightSide else -85.55
    side = case.faces(">X[0]").workplane(centerOption="ProjectedOrigin", origin=(82.77, yOffset, zOffset))
    inner = side.circle(2.5).extrude(-5,combine=False)
    outer = side.circle(3.7).extrude(-2,combine=False)
    return case.cut(inner).cut(outer)

def cutUsbCConnector(case):
    zOrigin = BOTTOM_PLATE_HEIGHT + PCB_BOTTOM_SPACING + 2.5
    workplane = case.faces("<Y[2]").workplane(centerOption="ProjectedOrigin", origin=(92.41, -41.75, zOrigin))
    inner = workplane.rect(9, 3.5).extrude(-6, combine=False).edges("-Y").fillet(1.7)
    outer = workplane.rect(12, 7).extrude(-2.5, combine=False).edges("-Y").fillet(3.4)
    return case.cut(inner).cut(outer)

def cutMountingHoles(case):
    for hole in HOLES:
        case = cutMountingHole(case, hole[0], diameter = hole[1], depth = hole[2])
    return cutMountingHole(case, (121.227683, 58.4)) #TrackPoint

def cutMountingHole(case, pnt, diameter = 2, depth = 0):
    defaultDepth = BOTTOM_PLATE_HEIGHT + PCB_BOTTOM_SPACING + PCB_HEIGHT + SWITCH_PLATE_HEIGHT
    d = defaultDepth if depth == 0 else depth
    return case.faces("<Z").workplane().moveTo(pnt[0], pnt[1]).hole(diameter, d)

def createBaseCase():
    height = SWITCH_BORDER_HEIGHT\
            + SWITCH_PLATE_HEIGHT\
            + PCB_HEIGHT\
            + PCB_BOTTOM_SPACING\
            + BOTTOM_PLATE_HEIGHT
    return createCaseOutline()\
        .faces()\
        .extrude(height)

## Wired Case (right)

In [6]:
caseWiredRight = createBaseCase()
caseWiredRight = extrudeDisplayCover(caseWiredRight).edges().fillet(2)
caseWiredRight = cutPcb(caseWiredRight)
caseWiredRight = cutKeycapsBorder(caseWiredRight)
caseWiredRight = cutSwitchHoles(caseWiredRight)
caseWiredRight = cutBottomPlate(caseWiredRight)
caseWiredRight = cutAcrylicGlassWindow(caseWiredRight)
caseWiredRight = cutAudioJack(caseWiredRight, rightSide=True)
caseWiredRight = cutMcu(caseWiredRight)
caseWiredRight = cutUsbCConnector(caseWiredRight)
caseWiredRight = cutMountingHoles(caseWiredRight)
show(caseWiredRight)

## Bottom Plate (right)

In [7]:
bottomPlateRight = createBottomPlate()

for hole in HOLES:
    bottomPlateRight = bottomPlateRight.faces("<Z[1]")\
        .workplane(centerOption="ProjectedOrigin", origin=(hole[0][0], -hole[0][1], 0))\
        .cskHole(2, 3, 82, depth=None)
    
show(bottomPlateRight)

In [8]:
all = (cq.Assembly()\
       .add(caseWiredRight, name="case", color=cq.Color("paleturquoise3"))\
       .add(bottomPlateRight, name="plate", color=cq.Color("paleturquoise4"))
      )
show(all)

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.33s


In [9]:
exporters.export(caseWiredRight, 'case_wired_right.stl')
exporters.export(bottomPlateRight, 'bottomPlate_right.stl')